In [4]:
#import python extensions and install pandas GBQ reader#
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import sys

In [5]:
# provide credential, service file, and client#
service_path = "/Users/peterkirgis/Downloads/"
service_file = 'wedge-project-peterkirgis-1e7b528def57.json' # this is your authentication information  
gbq_proj_id = 'wedge-project-peterkirgis'  # change this to your project_id
gbq_dataset_id = 'WedgeData2' # and change this to your data set ID

private_key = service_path + service_file

credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [7]:
#run query against transaction records that takes all data from 200 owners#
ownersample = """SELECT
              *
         FROM  `wedge-project-peterkirgis.WedgeData2.*`
LEFT OUTER JOIN (SELECT
              card_no as Owner, RAND() as randomidentifier
         FROM  
             `wedge-project-peterkirgis.WedgeData2.*`
         GROUP BY card_no
         ORDER BY card_no asc)
ON card_no=Owner
WHERE randomidentifier > 0.4 and randomidentifier < 0.41"""

table_id = "wedge-project-peterkirgis.WedgeData.owner_sample"

job_config = bigquery.QueryJobConfig(destination=table_id)

# And we execute queries with `client.query`
query_job = client.query(
        ownersample,
        job_config=job_config)
query_job.result()

In [8]:
#create DataFrame to read query output#
df = pd.read_gbq(ownersample, project_id=gbq_proj_id)

Downloading: 100%|██████████| 546836/546836 [03:14<00:00, 2810.27rows/s]


In [9]:
#write DataFrame to local textfile#
df.to_csv('owner_sample', sep = ",", header = True, index = False)